<a href="https://colab.research.google.com/github/classysai/sentiment-Analysis-by-web-scrapping/blob/main/sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vaderSentiment

     |████████████████████████████████| 125 kB 14.3 MB/s 


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs=SentimentIntensityAnalyzer()
text="Packet was totally damaged & ph is in dead condition, also there is no seal on the box , attached video for your reference"
vs.polarity_scores(text)

{'compound': -0.8655, 'neg': 0.316, 'neu': 0.684, 'pos': 0.0}

#web scrapping

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

In [ ]:
url="https://inshorts.com/en/read/sports"
news_data=[]
news_catagory=url.split("/")
news_catagory

['https:', '', 'inshorts.com', 'en', 'read', 'sports']

In [ ]:
data=requests.get(url)
soup=BeautifulSoup(data.content)
print(soup)

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<style>
    /* The Modal (background) */
    .modal_contact {
        display: none; /* Hidden by default */
        position: fixed; /* Stay in place */
        z-index: 8; /* Sit on top */
        left: 0;
        top: 0;
        width: 100%; /* Full width */
        height: 100%;
        overflow: auto; /* Enable scroll if needed */
        background-color: rgb(0,0,0); /* Fallback color */
        background-color: rgba(0,0,0,0.4); /* Black w/ opacity */
    }

    /* Modal Content/Box */
    .modal-content {
        background-color: #fefefe;
        margin: 15% auto;
        padding: 20px !important;
        padding-top: 0 !important;
        /* border: 1px solid #888; */
        text-align: center;
        position: relative;
        border-radius: 6px;
    }

    /* The Close Button */
    .close {
      left: 90%;
      color: #aaa;
      float: right;
      font-size: 28px;
      font-weight: bold;
    /* positio

In [18]:
urls=["https://inshorts.com/en/read/sports",
      "https://inshorts.com/en/read/politics",
      "https://inshorts.com/en/read/automobile"]
def build_dataset(urls):
  news_data=[]
  for url in urls:
    news_catagory=url.split("/")[-1]
    data=requests.get(url)
    soup=BeautifulSoup(data.content)

    news_articles=[{"news_headline":headline.find("span",attrs={"itemprop":"headline"}).string,
                    "news_article":article.find("div",attrs={"itemprop":"articleBody"}).string,
                    "news_category":news_catagory}
                   

                  for headline,article in zip(soup.find_all("div",class_=["news-card-title news-right-box"]),
                                              soup.find_all("div",class_=["news-card-content news-right-box"]))]
                    
    news_articles=news_articles[0:30]
    news_data.extend(news_articles)
  df=pd.DataFrame(news_data)
  df=df[["news_headline","news_article","news_category"]]
  return df

In [19]:
df=build_dataset(urls)
df.head()

,news_headline,news_article,news_category
0,How does the Tokyo Olympics medal tally look l...,Japan are on top of the medal tally with 13 go...,sports
1,Sri Lanka end their four-match losing streak i...,Sri Lanka defeated India by four wickets in th...,sports
2,Priyanka shares old interview with Biles after...,After four-time Olympic gold medallist Simone ...,sports
3,World number one archer Deepika reaches women'...,World number one Deepika Kumari advanced to ro...,sports
4,"NASA shares pic of Tokyo taken from space, say...",NASA took to Instagram to share a picture of l...,sports


In [ ]:
df.shape

(73, 3)

In [ ]:
df.to_csv("news_data.csv",index=False)

In [57]:
import nltk
nltk.download("stopwords")
stopword_list=nltk.corpus.stopwords.words("english")
stopword_list.remove("no")
stopword_list.remove("not")
len(stopword_list)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


177

In [ ]:
#remove html_tag
def html_tag(text):
  soup=BeautifulSoup(text,"html.parser")
  new_text=soup.get_text()
  return new_text
html_tag("<html><head2> hello can i know who's this </head2></html>")

" hello can i know who's this "

In [25]:
#Expand contractions
!pip install contractions
import contractions
def con(text):
  expand=contractions.fix(text)
  return expand
con("i'll kill you")

'I will kill you'

In [32]:
#remove the special characters
import re   #regular expression
def remove_special_character(text):
  pattern=r"[^A-Za-z0-9\s]"
  text=re.sub(pattern,"",text)
  return text
remove_special_character("heyy! you. who are you? *are you sk!@#$")

'heyy you who are you are you sk'

In [58]:
#stop words
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()
def remove_stopwords(text):
  tokens=tokenizer.tokenize(text)
  tokens=[token.strip() for token in tokens]
  filtered_tokens=[token for token in tokens if token not in stopword_list]
  filtered_text=" ".join(filtered_tokens)
  return filtered_text
remove_stopwords("The is the product is not good")

'The product not good'

In [42]:
 #1.lower case
 #2.remove html tags
 #3.expand contraction (meaning)
 #4.remove special character
 #5.reomove stop words

In [59]:
df.news_headline=df.news_headline.apply(lambda x:x.lower())
df.news_article=df.news_article.apply(lambda x:x.lower())


In [60]:
df.news_headline=df.news_headline.apply(html_tag)
df.news_article=df.news_article.apply(html_tag)

In [61]:
df.news_headline=df.news_headline.apply(con)
df.news_article=df.news_article.apply(con)

In [62]:
df.news_headline=df.news_headline.apply(remove_special_character)
df.news_article=df.news_article.apply(remove_special_character)

In [63]:
df.news_headline=df.news_headline.apply(remove_stopwords)
df.news_article=df.news_article.apply(remove_stopwords)

In [64]:
df

,news_headline,news_article,news_category
0,tokyo olympics medal tally look like todays ev...,japan top medal tally 13 gold medals four silv...,sports
1,sri lanka end fourmatch losing streak t20i cri...,sri lanka defeated india four wickets last sec...,sports
2,priyanka shares old interview biles gymnast wi...,fourtime olympic gold medallist simone biles w...,sports
3,world number one archer deepika reaches womens...,world number one deepika kumari advanced round...,sports
4,nasa shares pic tokyo taken space says olympic...,nasa took instagram share picture litup japans...,sports
...,...,...,...
68,tata motors q1 net loss narrows 4450 cr revenu...,tata motors narrowed consolidated net loss 445...,automobile
69,alphabets waymo asks uk govt not limit autonom...,waymo selfdriving car startup owned googles pa...,automobile
70,porsche must pay 47 mn breach tax filings duty...,porsche must pay fine 47 million 40 million br...,automobile
71,global chip shortage dent car sales 2021s 2nd ...,mercedesbenz maker daimler said global shortag...,automobile


In [70]:
df["compound"]=df["news_article"].apply(lambda x: vs.polarity_scores(x)["compound"])
df


,news_headline,news_article,news_category,compound
0,tokyo olympics medal tally look like todays ev...,japan top medal tally 13 gold medals four silv...,sports,0.6369
1,sri lanka end fourmatch losing streak t20i cri...,sri lanka defeated india four wickets last sec...,sports,-0.4767
2,priyanka shares old interview biles gymnast wi...,fourtime olympic gold medallist simone biles w...,sports,-0.4082
3,world number one archer deepika reaches womens...,world number one deepika kumari advanced round...,sports,-0.5423
4,nasa shares pic tokyo taken space says olympic...,nasa took instagram share picture litup japans...,sports,0.2960
...,...,...,...,...
68,tata motors q1 net loss narrows 4450 cr revenu...,tata motors narrowed consolidated net loss 445...,automobile,0.0000
69,alphabets waymo asks uk govt not limit autonom...,waymo selfdriving car startup owned googles pa...,automobile,0.1695
70,porsche must pay 47 mn breach tax filings duty...,porsche must pay fine 47 million 40 million br...,automobile,0.3027
71,global chip shortage dent car sales 2021s 2nd ...,mercedesbenz maker daimler said global shortag...,automobile,-0.5423


In [72]:
df.to_csv("news_data with compound_score.csv",index=False)